In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tomotok.core.phantoms import gauss_iso
from tomotok.core.geometry import generate_los, sparse_line_3d, RegularGrid
from tomotok.core.derivative import compute_iso_dmats
from tomotok.core.inversions import SvdFastAlgebraic, GevFastAlgebraic

In [ ]:
grid = RegularGrid(50, 100, (.2, .7), (-.5, .5))

In [ ]:
phantom = gauss_iso(grid.nr, grid.nz, cen=.3, amp=100)

In [ ]:
plt.figure()
plt.imshow(phantom, extent=grid.extent, origin='lower')
plt.colorbar(label='Emissivity [-]')
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.show()

In [ ]:
num = 20
ss = []
es = []
s1, e1 = generate_los(num=(num, 1), fov=(70, 0), pinhole=(1, 0, 0), axis=(-1, 0, 0))
s2, e2 = generate_los(num=(num, 1), fov=(35, 0), pinhole=(.4, 0, .8), axis=(.1, 0, -1), elong=1.5)
s3, e3 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.75, 0, -.4), axis=(-1, 0, 1), elong=1.5)
s4, e4 = generate_los(num=(num, 1), fov=(50, 0), pinhole=(.75, 0, .4), axis=(-1, 0, -1), elong=1.5)
ss.append(s1)
es.append(e1)
ss.append(s2)
es.append(e2)
ss.append(s3)
es.append(e3)
es.append(e4)
ss.append(s4)
s = np.concatenate(ss, 0)
e = np.concatenate(es, 0)

In [ ]:
rch = np.empty((s.shape[0], 2))
zch = np.empty((s.shape[0], 2))
rch[:, 0] = s[:, 0]
rch[:, 1] = e[:, 0]
zch[:, 0] = s[:, 2]
zch[:, 1] = e[:, 2]

In [ ]:
gmat = sparse_line_3d(rch, zch, grid, rmin=.2)
gmat = gmat.toarray()

In [ ]:
plt.figure()
plt.imshow(gmat.sum(0).reshape(grid.shape), extent=grid.extent, origin='lower')
plt.colorbar(label='Length [-]')
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.show()

In [ ]:
sig = gmat.dot(phantom.flatten())

In [ ]:
plt.figure()
plt.plot(sig, '+')
plt.xlabel('Channel [-]')
plt.ylabel('Signal [-]')
plt.show()

In [ ]:
data = sig.reshape(1, -1)  # data should have shape (#timeslices, #channels/pixels)

In [ ]:
errors = (data + data.max() ) / 2 * .05
# errors = .001

In [ ]:
svd = SvdFastAlgebraic()  # no sparse optimization
gev = GevFastAlgebraic()  # has some sparse optimization

In [ ]:
dmats = compute_iso_dmats(grid, derivatives=[1,2])

In [ ]:
sout = svd(data, gmat, dmats, errors, method='median')

In [ ]:
plt.figure()
plt.title('SVD')
plt.imshow(sout[0].reshape(grid.shape), origin='lower', extent=grid.extent)
plt.colorbar(label='Emissivity [-]')
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.show()

In [ ]:
gout = gev(data, gmat, dmats, errors)

In [ ]:
plt.figure()
plt.title('GEV')
plt.imshow(gout[0].reshape(grid.shape), origin='lower', extent=grid.extent)
plt.colorbar(label='Emissivity [-]')
plt.xlabel('R [-]')
plt.ylabel('z [-]')
plt.show()